# Stocks
This notebook shows how to treat the stock prices and make some simple analyses of financial time series.

## How to read and handle a csv file
Here we open a csv (comma separated values) file containing the stock prices (in this case, daily close prices). We first get the data from the repository.

In [ ]:
from sociophysicsDataHandler import SociophysicsDataHandler

student_config = True

file_target = 'asdz/platform2.2/20200428/ASDZ_Perron2.2_2020042815_trajectorie.parquet' 

if student_config:
    dh = SociophysicsDataHandler()
    dh.fetch_prorail_data_from_path(file_target)
else:
    webdav_basepath='/Crowdflow (Projectfolder)/ProRail_USE_LL_data'
    dh = SociophysicsDataHandler(basepath=webdav_basepath)
    
    dh.fetch_prorail_data_from_path(file_target)
                           # ,basepath=webdav_basepath)

In [ ]:
import pandas as pd
dh.fetch_econophysics_data_from_path("econophysics/prices/daily_close_prices.csv")
prices = dh.df
prices.index = pd.to_datetime(prices.index) # to be sure that the index is in the pandas DateTime format
print(prices)

## Plot stock daily prices between a certain time range
Define the time range and the stock we want to show, and plot the corresponding close prices.

In [ ]:
import matplotlib.pyplot as pl
stock = 'GME'
start_month = '2018-01'
end_month = '2022-07'
prices_toPlot = prices[[stock]][start_month:end_month]
prices_toPlot.plot()
pl.xlabel('Time', fontsize=14)
pl.ylabel('Price (USD)', fontsize=14)
pl.title(stock + ', close price', fontsize=14)
pl.show()

## Daily returns
Calculate daily returns and their statistics. Daily returns are percentage changes between consecutive prices. For example, if yesterday's price was 150 USD and today's price is 152 USD, then the daily return is (152-150)/150 = 0.013, i.e. 1.3%

In [ ]:
rets = prices.pct_change()
print('The mean daily returns are:')
print(rets.mean())
print('The standard deviation of the daily returns are:')
print(rets.std())
# standard deviations of daily returns show the level of fluctuations of the stocks;
# higher standard deviation implies higher stock volatility (see "Calculate stock volatility")

## Filtering and grouping
We can filter the returns dataframe based on some condition (for example, the magnitude of the returns).

In [ ]:
return_threshold = 2 # we will select only the days with daily return larger than 2%
stock = 'AAPL'
rets_percent = 100*rets #  transform returns into percent returns to increase readability
rets_percent_large = rets_percent[[stock]][rets_percent > return_threshold] # in the square brackets, we set the condition
rets_percent_small = rets_percent[[stock]][rets_percent < -return_threshold]
rets_percent_large.dropna()

## Calculate stock volatility
From the stock prices and returns we can calculate the volatility, i.e. the average fluctuations of the stock in the last n days. The volatility represents the risk associated to the financial object: the higher the volatility, the riskier the security. In mathematical terms, it is nothing but the weighted variance of the stock returns multiplied by a factor, as shown in the function below.

In [ ]:
import numpy as np
import datetime

def volaCalc2(x, w): # x is your pandas dataframe showing the close-close returns and w is a vector of weights
    # in this case, we calculate the volatility on 65 trading days, and we weight the last 5 days twice w.r.t. to the others:
    w = np.append(np.repeat(1, 60),np.repeat(2, 5)) 
    ttt=datetime.datetime.now() # check time
    
    n_col = x.shape[1]
    n_row = x.shape[0]
    n_w = len(w)
    s_w = sum(w)
    tmp_mean = np.tile(np.nan, (n_row, n_col))
    vola = np.tile(np.nan, (n_row, n_col))
    x_array = x.values # transform pandas into array
    
    for j in range(0,n_col): # loop on columns
        tmpcol=x_array[:,j];
        for i in range(0,n_row): # loop on rows
            if i >= n_w:
                tmp_mean[i,j] = sum((w*tmpcol[i+1-n_w:i+1]))/s_w
                vola[i,j] = sum(w*(tmpcol[i+1-n_w:i+1]-tmp_mean[i,j])*(tmpcol[i+1-n_w:i+1]-tmp_mean[i,j]))/(s_w-1) #weighted variance
                
    vola = pd.DataFrame(vola, index = x.index, columns=x.columns)
    print(' total time: '+str((datetime.datetime.now() - ttt).seconds + (datetime.datetime.now() - ttt).microseconds/1E6) + ' s')        
    return(vola)

weights = np.append(np.repeat(1, 60),np.repeat(2, 5))
volatility_daily = np.sqrt(volaCalc2(rets, weights)) # compute the square root of the variance
# this is the daily volatility; usually, though, the volatility is presented in annualized terms 
# assuming there are 252 trading days in a year, we therefore multiply the daily volatility by the square root of 252:
volatility = np.sqrt(252)*volatility_daily
print('The annualized volatility of the last 5 days of the dataset is:')
print(100*volatility.tail())

## Time zones
One of the crucial aspects, when dealing with financial time series, is paying attention to the time zones in which the data are expressed. This is important if we want to correctly match the Reddit database (where times are expressed in UTC a.k.a. GMT) with the stock database (expressed in New York time). Here we show how to transform the latter into UTC time. This issue is only present when dealing with intraday prices (in this case, at hourly resolution).

In [ ]:
dh.fetch_econophysics_data_from_path("econophysics/prices/hourly_prices.csv")
prices_hour = dh.df
prices_hour.index = pd.to_datetime(prices_hour.index) # to be sure that the index is in the pandas DateTime format
print(prices_hour)

In [ ]:
# the times you see in the index of prices_hour are expressed in New York time (American Eastern Time) 
# American Eastern Time is defined as UTC-5 in autumn and winter, and UTC-4 in spring and summer (daylight saving)
import datetime as dt
from dateutil import tz # library to treat timezones
NYC = tz.gettz('America/New_York') # define the New York timezone

# we begin with a single time as an example; let's take the 13th entry in the prices dataframe:
i = 13
dat = str(prices_hour.index[i].date())
print('The considered date is:', dat)
print('In particular, we are considering the index:', prices_hour.index[i], 'in New York time')

In [ ]:
# let's compute the offset between New York time and UTC (the time we want to use) on that date:
dt1 = dt.datetime(int(dat[0:4]), int(dat[5:7]),int(dat[8:10]), tzinfo=NYC)
UTC_lag = dt1.utcoffset() / dt.timedelta(hours=1)
print('On', dat, 'the offset is', UTC_lag, 'hours')

In [ ]:
# now we can actually transform the time index into UTC:
new_index = prices_hour.index[i] - pd.Timedelta(hours=UTC_lag)
print('The 13th index in New York time is:', prices_hour.index[i])
print('The 13th index in UTC time is:', new_index)

In [ ]:
# if we do the same for a winter day, then we see that the offset is in that case 5 hours
i = 2000
dat = str(prices_hour.index[i].date())
dt1 = dt.datetime(int(dat[0:4]), int(dat[5:7]),int(dat[8:10]), tzinfo=NYC)
UTC_lag = dt1.utcoffset() / dt.timedelta(hours=1)
print('On', dat, 'the offset is', UTC_lag, 'hours')

In [ ]:
# we can therefore transform the whole dataframe based on this rule:
prices_hour

In [ ]:
new_index_list = []
for i in range(len(prices_hour)):
    # for each index, make the previous index transformation:
    old_index = prices_hour.index[i]
    dat = str(old_index.date())
    dt1 = dt.datetime(int(dat[0:4]), int(dat[5:7]),int(dat[8:10]), tzinfo=NYC)
    UTC_lag = dt1.utcoffset() / dt.timedelta(hours=1)
    new_index = prices_hour.index[i] - pd.Timedelta(hours=UTC_lag)
    # attach the UTC index to the new index list:
    new_index_list.append(new_index)
    
# set the new UTC index to the prices_hour dataframe:
prices_hour.index = new_index_list

In [ ]:
prices_hour